In [1]:
import os
import sktime
from sktime.datasets import load_from_tsfile_to_dataframe
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import sys
sys.path.append("../../")
from src.Helpers import append_to_csv

def plot_model_history(history, epochs, path=None):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']
  print(f'Highest Validation Accuracy: {np.max(val_accuracy)}')
  eps = range(epochs)

  plt.figure()
  plt.plot(eps, loss, 'r', label='Training loss')
  plt.plot(eps, val_loss, 'b', label='Validation loss')
  plt.title('Training and Validation Loss')
  plt.xlabel('Epoch')
  plt.ylabel('Loss Value')
  plt.ylim([0, 2])
  plt.legend()
  if path:
      plt.savefig(os.path.join(path, 'train_and_valid_loss.png'))
  plt.show()

  plt.figure()
  plt.plot(eps, accuracy, 'r', label='Training accuracy')
  plt.plot(eps, val_accuracy, 'b', label='Validation accuracy')
  plt.title('Training and Validation accuracy')
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy Value')
  plt.ylim([0, 1])
  plt.legend()
  if path:
      plt.savefig(os.path.join(path, 'train_and_valid_accuracy.png'))
  plt.show()

In [4]:
path_persist_results = "./training_res.csv"
columns = ["dataset_name", "model_name", "test_loss", "test_acc", "confusion_matrix", "history"]
append_to_csv(path_persist_results, columns)

# Load data

In [2]:
dataset_path = "../../datasets/abnormal_heartbeat"
path_test_ds = os.path.join(dataset_path, "AbnormalHeartbeat_TEST.ts")
path_train_ds = os.path.join(dataset_path, "AbnormalHeartbeat_TRAIN.ts")

In [3]:
def prepare_train_data(df):
    prepared = list()
    for i in range(df.size):
        prepared.append(df.iloc[i].to_numpy()[0].to_numpy())
    return np.array(prepared, dtype=np.float64)

def load_numpy_array_from_ts(path_to_file):
    x, y = load_from_tsfile_to_dataframe(path_to_file)
    return prepare_train_data(x), np.array(y, dtype=np.float64)

In [4]:
x_test, y_test = load_numpy_array_from_ts(path_test_ds)
x_train, y_train = load_numpy_array_from_ts(path_train_ds)

In [5]:
x_test

array([[-0.00048828, -0.00247192, -0.00375366, ..., -0.01000977,
        -0.0098877 , -0.00970459],
       [ 0.01397705,  0.02963257,  0.02502441, ...,  0.04724121,
         0.04833984,  0.04873657],
       [ 0.06552124,  0.1074524 ,  0.05529785, ...,  0.03118896,
         0.02786255,  0.02401733],
       ...,
       [ 0.01196289,  0.03741455,  0.05322266, ...,  0.03417969,
         0.03375244,  0.03338623],
       [ 0.00045776,  0.02279663,  0.04455566, ...,  0.00469971,
         0.00289917,  0.00213623],
       [ 0.05514526,  0.1166687 ,  0.1042175 , ..., -0.04534912,
        -0.0453186 , -0.04568481]])

In [6]:
y_test

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1.])

# Prepare data

# Train models

In [8]:
from src.ModelBuilder import get_MLP, get_MCDCNN, get_Time_CNN, get_FCN

In [9]:
x_train.shape

(204, 18530)

In [10]:
y_train.shape

(204,)

In [11]:
input_size = x_train.shape
num_columns = input_size[1]
output_size = 2

In [12]:
model_eval_results = dict()
training_history = dict()

In [ ]:
for get_model, model_name in zip([get_FCN, get_MCDCNN, get_Time_CNN, get_MLP], ["FCN", "MLP", "MCDCNN", "Time_CNN"]):
#for get_model, model_name in zip([get_MCDCNN, get_Time_CNN], ["MLP", "MCDCNN", "Time_CNN"]):
    model = get_model(num_columns, output_size)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    history = model.fit(x_train, y_train, batch_size=10, epochs=30, validation_split=0.2)
    test_loss, test_acc = model.evaluate(x_test, y_test)
    model_eval_results[model_name] = (test_loss, test_acc)
    training_history[model_name] = history

Epoch 1/30
17/17 [==============================] - 124s 7s/step - loss: 0.3746 - accuracy: 0.8957 - val_loss: 0.8531 - val_accuracy: 0.0000e+00
Epoch 2/30
17/17 [==============================] - 94s 5s/step - loss: 0.2923 - accuracy: 0.9141 - val_loss: 0.9809 - val_accuracy: 0.0000e+00
Epoch 3/30
17/17 [==============================] - 109s 6s/step - loss: 0.2783 - accuracy: 0.9080 - val_loss: 1.1485 - val_accuracy: 0.0000e+00
Epoch 4/30
17/17 [==============================] - 113s 7s/step - loss: 0.2979 - accuracy: 0.9141 - val_loss: 1.1230 - val_accuracy: 0.0000e+00
Epoch 5/30
17/17 [==============================] - 97s 6s/step - loss: 0.2914 - accuracy: 0.9141 - val_loss: 1.3680 - val_accuracy: 0.0000e+00
Epoch 6/30
17/17 [==============================] - 85s 5s/step - loss: 0.2877 - accuracy: 0.9141 - val_loss: 1.4106 - val_accuracy: 0.0000e+00
Epoch 7/30
17/17 [==============================] - 86s 5s/step - loss: 0.2688 - accuracy: 0.9141 - val_loss: 1.4232 - val_accuracy: 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\tmozo\documents\master\semester 3\projekt deep learning\code\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\tmozo\AppData\Local\Temp\ipykernel_21480\2937934318.py", line 7, in <module>
    history = model.fit(x_train, y_train, batch_size=10, epochs=30, validation_split=0.2)
  File "c:\users\tmozo\documents\master\semester 3\projekt deep learning\code\venv\lib\site-packages\keras\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "c:\users\tmozo\documents\master\semester 3\projekt deep learning\code\venv\lib\site-packages\keras\engine\training.py", line 1650, in fit
    tmp_logs = self.train_function(iterator)
  File "c:\users\tmozo\documents\master\semester 3\projekt deep learning\code\venv\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 150, in error_handler
    return fn(

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "c:\users\tmozo\documents\master\semester 3\projekt deep learning\code\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\tmozo\AppData\Local\Temp\ipykernel_21480\2937934318.py", line 7, in <module>
    history = model.fit(x_train, y_train, batch_size=10, epochs=30, validation_split=0.2)
  File "c:\users\tmozo\documents\master\semester 3\projekt deep learning\code\venv\lib\site-packages\keras\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "c:\users\tmozo\documents\master\semester 3\projekt deep learning\code\venv\lib\site-packages\keras\engine\training.py", line 1650, in fit
    tmp_logs = self.train_function(iterator)
  File "c:\users\tmozo\documents\master\semester 3\projekt deep learning\code\venv\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 150, in error_handler
    return fn(

In [13]:
pd.DataFrame.from_dict(model_eval_results, orient='index',
                       columns=["test_loss", "test_acc"])

ERROR! Session/line number was not unique in database. History logging moved to new session 123


,test_loss,test_acc


| |test_loss|	test_acc|
-----------|-----|-----------------|-----------------|
| FCN	      |0.671740|	0.721951|
| MLP	      |1.067936|	0.731707|
| MCDCNN	   |0.667893|	0.731707|
| Time_CNN	 |5.187996|	0.721951|